In [1]:
# import libraries
from sentence_transformers import SentenceTransformer
from collections import Counter
import requests
import json
import os
from tqdm import tqdm
import pandas as pd
import json

# Change to the parent directory of the notebook
os.chdir('..')

In [ ]:
# EMBEDDINGS_MODEL = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()
# df = pd.read_csv("Data/dataset.csv", index_col=0)
# print("generating embeddings...")
# # Generate embeddings for each row in the 'tweet_text' column
# tqdm.pandas()

# # Generate embeddings with progress tracking
# df['X_train'] = df['tweet_text'].progress_apply(lambda text: EMBEDDINGS_MODEL.encode(text))


In [2]:
import pickle

# Load using pickle
with open("Data/dataset_mit_embeddings.pkl", "rb") as f:
    df = pickle.load(f)


df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20148 entries, 0 to 20147
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   post_id             20148 non-null  object
 1   tweet_text          20148 non-null  object
 2   key_features        20148 non-null  object
 3   target              16230 non-null  object
 4   label               20148 non-null  object
 5   annotator_1_label   20148 non-null  object
 6   annotator_1_target  13021 non-null  object
 7   annotator_2_label   20148 non-null  object
 8   annotator_2_target  13151 non-null  object
 9   annotator_3_label   20148 non-null  object
 10  annotator_3_target  13080 non-null  object
 11  X_train             20148 non-null  object
 12  label_encoded       20148 non-null  int64 
dtypes: int64(1), object(12)
memory usage: 2.2+ MB


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Convert 'X_train' column to a list of embeddings
X = df['X_train'].tolist()
y = df['label_encoded'].tolist()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Use 'multi:softmax' for multi-class classification
    num_class=len(label_encoder.classes_),  # Number of classes
    eval_metric='mlogloss',  # Multi-class log loss
    use_label_encoder=False  # Avoid warnings about label encoding
)

# Convert data to proper format if needed
# X_train and X_test are lists of lists (embeddings), so they're ready for XGBoost.

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:50:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.6278

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.69      0.68      0.68      1285
      normal       0.64      0.75      0.69      1610
   offensive       0.51      0.40      0.45      1135

    accuracy                           0.63      4030
   macro avg       0.61      0.61      0.61      4030
weighted avg       0.62      0.63      0.62      4030

